<a href="https://colab.research.google.com/github/Glitch0000/Using_multible_GPUs_One_Device_Strategy_Messy_vs_Clean_Room/blob/main/Using_multible_GPUs_One_Device_Strategy_Messy_vs_Clean_Room.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# One Device Strategy an example to classify Messy vs Clean Room

In this example we will use [One Device Strategy](https://www.tensorflow.org/api_docs/python/tf/distribute/OneDeviceStrategy). This is typically used to deliberately test the code on a single device. This can be used before switching to a different strategy that distributes across multiple devices.

 The dataset used is [Messy vs Clean Room](https://www.kaggle.com/cdawn1/messy-vs-clean-room).

## Imports

In [1]:
import tensorflow as tf
import tensorflow_hub as hub
import tensorflow_datasets as tfds

tfds.disable_progress_bar()

In [2]:
!unzip  /content/archive.zip #unzip data in the directory
#IIn colab it is not possible to include the folders of the images before unziping them

In [3]:
img_size= 420

In [4]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

# All images will be rescaled by 1./255
train_datagen = ImageDataGenerator(
      rescale=1./255,
      rotation_range=40,
      width_shift_range=0.2,
      height_shift_range=0.2,
      shear_range=0.2,
      zoom_range=0.2,
      fill_mode='nearest',
      horizontal_flip=True
      )

validation_datagen = ImageDataGenerator(rescale=1/255)

# Flow training images in batches of 128 using train_datagen generator
train_generator = train_datagen.flow_from_directory(
        '/content/images/images/train',  # This is the source directory for training images
        target_size=(img_size, img_size),  # All images will be resized to img_size x img_size
        batch_size=16,
        #  class_mode='binary'in case we use binary_crossentropy loss, we need binary labels  
        class_mode= 'categorical')

# Flow validation images in batches of batch_size
validation_generator = validation_datagen.flow_from_directory(
        '/content/images/images/val',  # This is the source directory for validation images
        target_size=(img_size, img_size),  
        batch_size=8,
        class_mode= 'categorical')

Found 192 images belonging to 2 classes.
Found 20 images belonging to 2 classes.


## Define the Distribution Strategy

We can list available devices in our machine and specify a device type. This allows us to verify the device name to pass in `tf.distribute.OneDeviceStrategy()`.

In [5]:
# choose a device type such as CPU or GPU
devices = tf.config.list_physical_devices('GPU')
print(devices[0])

# the name will look something like "/physical_device:GPU:0"
# Just take the GPU:0 part and use that as the name
gpu_name = "GPU:0"

# define the strategy and pass in the device name
one_strategy = tf.distribute.OneDeviceStrategy(device=gpu_name)

PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')


## Define and Configure the Model

As with other strategies, setting up the model requires minimal code changes. Let's first define a utility function to build and compile the model.

In [6]:
# tells if we want to freeze the layer weights of our feature extractor during training
do_fine_tuning = False

In [7]:
def build_and_compile_model():
  
    # defining the model
    model = tf.keras.models.Sequential([
    # Note the input shape is the desired size of the image img_size x img_size with 3 bytes color
    # This is the first convolution
    tf.keras.layers.Conv2D(16, (3,3), activation='relu', input_shape=(img_size, img_size, 3)),
    tf.keras.layers.MaxPooling2D(2, 2),
    # The second convolution
    tf.keras.layers.Conv2D(32, (3,3), activation='relu'),
    tf.keras.layers.MaxPooling2D(2,2),
    # The third convolution
    tf.keras.layers.Conv2D(64, (3,3), activation='relu'),
    tf.keras.layers.MaxPooling2D(2,2),
    tf.keras.layers.Flatten(),
    # 512 neuron hidden layer
    tf.keras.layers.Dense(512, activation='relu'),
   
    tf.keras.layers.Dense(2, activation='softmax')# number of classes, in case we have binary classification:(1, activation='sigmid')
              ])

    # display summary
    model.summary()

    # configure the optimizer, loss and metrics
    optimizer = tf.keras.optimizers.SGD(lr=0.002, momentum=0.9) if do_fine_tuning else 'adam'
    model.compile(optimizer=optimizer,
                loss='categorical_crossentropy',#loss='sparse_categorical_crossentropy',
                metrics=['accuracy'])

    return model

Note: here we faced data mismatch due to the reason of using sparse_categorical_crossentropy as a loss function instead of categorical_crossentropy.

Now we call the function under the strategy scope. This places variables and computations on the device we specified earlier.

In [8]:
# build and compile under the strategy scope
with one_strategy.scope():
    model = build_and_compile_model()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 418, 418, 16)      448       
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 209, 209, 16)      0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 207, 207, 32)      4640      
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 103, 103, 32)      0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 101, 101, 64)      18496     
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 50, 50, 64)        0         
_________________________________________________________________
flatten (Flatten)            (None, 160000)            0

`model.fit()` can be run as usual.

In [10]:
EPOCHS = 15
hist = model.fit(train_generator,
                 epochs=EPOCHS,
                 validation_data=validation_generator)

Epoch 1/15
12/12 [==============================] - 9s 745ms/step - loss: 0.5735 - accuracy: 0.7396 - val_loss: 0.5707 - val_accuracy: 0.6500
Epoch 2/15
12/12 [==============================] - 9s 740ms/step - loss: 0.4875 - accuracy: 0.7917 - val_loss: 0.4145 - val_accuracy: 0.8000
Epoch 3/15
12/12 [==============================] - 9s 732ms/step - loss: 0.4885 - accuracy: 0.7760 - val_loss: 0.3310 - val_accuracy: 0.8000
Epoch 4/15
12/12 [==============================] - 9s 733ms/step - loss: 0.5079 - accuracy: 0.7708 - val_loss: 0.4326 - val_accuracy: 0.8000
Epoch 5/15
12/12 [==============================] - 9s 738ms/step - loss: 0.4945 - accuracy: 0.7656 - val_loss: 0.4516 - val_accuracy: 0.7500
Epoch 6/15
12/12 [==============================] - 9s 743ms/step - loss: 0.5550 - accuracy: 0.6927 - val_loss: 0.3993 - val_accuracy: 0.9500
Epoch 7/15
12/12 [==============================] - 9s 736ms/step - loss: 0.5283 - accuracy: 0.7188 - val_loss: 0.5713 - val_accuracy: 0.6500
Epoch 

Since everything is working correctly, we can switch to a different device or a different strategy that distributes to multiple devices.